In [ ]:
!pip install -q medmnist scikit-learn

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from medmnist import PathMNIST
import medmnist
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from train_cnn import CNNBaseline
from train_nca import NCA

from google.colab import drive
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
nca = NCA().to(device)
nca.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_nca_pathmnist.pth"))
nca.eval()

cnn = CNNBaseline().to(device)
cnn.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_cnn_pathmnist.pth"))
cnn.eval()

In [ ]:
def get_loader(size, batch_size=64):
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = PathMNIST(split="val", size=size, download=True, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
@torch.no_grad()
def evaluate(model, loader, name="Model"):
    all_preds, all_labels = [], []

    for x, y in loader:
        x, y = x.to(device), y.squeeze()
        out = model(x)
        pred = out.argmax(dim=1).cpu()
        all_preds.extend(pred.numpy())
        all_labels.extend(y.numpy())

    print(f"\n{name} Results")
    print(confusion_matrix(all_labels, all_preds))
    print(classification_report(all_labels, all_preds, digits=4))

In [ ]:
for size in [28, 64, 128, 224]:
    print(f"\n==============================")
    print(f"🖼️  Resolution: {size}x{size}")
    loader = get_loader(size)

    print("🔍 CNN:")
    evaluate(cnn, loader, f"CNN @ {size}")

    print("🔁 NCA:")
    evaluate(nca, loader, f"NCA @ {size}")